 TODOs (intentionally not implemented):
- `build_training_data`
- `class PINN`
- `model_loss`
- `train`


In [22]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import math
from typing import Tuple

import numpy as np
import torch
import torch.nn as nn


def build_training_data():
    """
    TODO : Implement the training-data construction.
    """

    raise NotImplementedError("TODO: implement build_training_data")


class PINN(nn.Module):
    def __init__():
        raise NotImplementedError("")
    
    def forward(self, x):
        raise NotImplementedError("")

        
        


def grad(outputs, inputs):  # compute derivative of outputs w.r.t. inputs 
    return torch.autograd.grad( 
        outputs,
        inputs,
        grad_outputs=torch.ones_like(outputs), 
        create_graph= True,  
        retain_graph=True,
    )[0]

def pde(u,x,t, nu ): 
    raise NotImplementedError("TODO: implement PDE residual computation")
def model_loss(
       ):
    """
    TODO : Implement PINN loss.

    """
    raise NotImplementedError("TODO: implement model_loss")






In [ ]:
# Run after implementing all TODOs:
#model = train()
#evaluate(model)
